In [1]:
# import libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate
from ISLP.models import sklearn_sm
from sklearn.model_selection import KFold
from sklearn.base import clone
from functools import partial

1.Validation Set Approach

In [2]:
# load data and split into training and test sets
data = load_data('Boston')
train_data, test_data = train_test_split(data, test_size=0.5, random_state=42)

In [3]:
# Fit polynomial models of degrees 1-4, calculate validation MSE for each, and report as a numpy array (rounded to 2 decimals)
y_train = train_data['medv']
X_train = train_data[['lstat']] 
y_test = test_data['medv']
X_test = test_data[['lstat']]  
mse_list = []

for degree in range(1, 5):
    # 1. Create polynomial features object
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    
    # 2. Create the polynomial design matrix for the training data
    X_poly_train = poly_features.fit_transform(X_train)
    
    # 3. Add a constant (intercept) for the statsmodels OLS model
    X_poly_train = sm.add_constant(X_poly_train)
    
    # 4. Transform the test data using the *same* polynomial features
    X_poly_test = poly_features.transform(X_test)
    X_poly_test = sm.add_constant(X_poly_test)
    
    # Fit model
    model = sm.OLS(y_train, X_poly_train).fit()
    
    # Predict on test set
    y_pred = model.predict(X_poly_test)
    
    # Calculate MSE
    mse = np.mean((y_test - y_pred) ** 2)
    mse_list.append(mse)

mse_array = np.round(np.array(mse_list), 2)
print("MSE for degrees 1 to 4:")
print(mse_array)

MSE for degrees 1 to 4:
[38.51 30.84 29.22 27.75]


2.Leave-One-Out Cross-Validation

In [2]:
# load College data
data2 = load_data('College')

In [19]:
# Implement LOOCV for polynomial degrees 1 through 5
cv_error = np.zeros(5) #array to store CV errors
H = np.array(data2['Room.Board']) #extract horsepower values
M = sklearn_sm(sm.OLS)  #create sklearn-compatible model
X = data2[["Room.Board"]]
Y = data2['Outstate']
for i, d in enumerate(range(1,6)): #loop over polynomial degrees 1 to 5
    X = np.power.outer(H, np.arange(d+1)) #create polynomial features
    M_CV = cross_validate(M, 
                          X,
                          Y,
                          cv=data2.shape[0]) #LOOCV
    cv_error[i] = np.mean(M_CV['test_score']) #average test MSE
cv_error #display LOOCV errors for polynomial degrees 1 to 5
cv_errors_array = np.round(np.array(cv_error), 2)
cv_errors_array

array([9291471.1 , 9255509.68, 9263314.52, 9269448.22, 9300815.64])

3.K-Fold Cross-Validation Comparison

In [16]:
# Implement 5-fold CV for polynomial degrees 1 through 3
cv5 = KFold(n_splits=5,
           shuffle=True,
           random_state=123)
cv_error_k5 = np.zeros(3) #array to store CV errors
H = np.array(data2['Room.Board']) #extract horsepower values
M = sklearn_sm(sm.OLS)  #create sklearn-compatible model
X, Y = data2.drop(columns=['Outstate']), data2['Outstate']
for i, d in enumerate(range(1,4)): #loop over polynomial degrees 1 to 5
    X = np.power.outer(H, np.arange(d+1)) #create polynomial features
    M_CV = cross_validate(M, 
                          X,
                          Y,
                          cv=cv5) #LOOCV
    cv_error_k5[i] = np.mean(M_CV['test_score']) #average test MSE
cv_errors_array_k5 = np.round(np.array(cv_error_k5), 3)
print("5-Fold CV errors for polynomial degrees 1 to 3:")
print(cv_errors_array_k5)

5-Fold CV errors for polynomial degrees 1 to 3:
[9377937.163 9345970.983 9385687.226]


In [12]:
# Implement 10-fold CV for polynomial degrees 1 through 3
cv10 = KFold(n_splits=10,
           shuffle=True,
           random_state=123)
cv_error_k10 = np.zeros(3) #array to store CV errors
H = np.array(data2['Room.Board']) #extract horsepower values
M = sklearn_sm(sm.OLS)  #create sklearn-compatible model
X, Y = data2.drop(columns=['Outstate']), data2['Outstate']
for i, d in enumerate(range(1,4)): #loop over polynomial degrees 1 to 5
    X = np.power.outer(H, np.arange(d+1)) #create polynomial features
    M_CV = cross_validate(M, 
                          X,
                          Y,
                          cv=cv10) #LOOCV
    cv_error_k10[i] = np.mean(M_CV['test_score']) #average test MSE
cv_errors_array_k10 = np.round(np.array(cv_error_k10), 3)
print("10-Fold CV errors for polynomial degrees 1 to 3:")
print(cv_errors_array_k10)

10-Fold CV errors for polynomial degrees 1 to 3:
[9315542.59  9281464.281 9293767.861]


4.Bootstrap Standard Errors

In [11]:
# Load the Default dataset
data3 = load_data('Default')

In [ ]:
# Function to compute alpha
def alpha_func(D, idx): #function to compute alpha
   cov_ = np.cov(D[['balance','income']].loc[idx], rowvar=False) #covariance matrix
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1])) #compute alpha 


In [4]:
# Bootstrap standard error function
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0): #function to compute bootstrap standard error
    rng = np.random.default_rng(seed) #random number generator
    first_, second_ = 0, 0 #initialize accumulators
    n = n or D.shape[0] #set n to number of observations if not provided
    for _ in range(B): #bootstrap iterations
        idx = rng.choice(D.index,  
                         n,
                         replace=True) #bootstrap sample indices
        value = func(D, idx) #compute statistic
        first_ += value #accumulate first moment
        second_ += value**2 #accumulate second moment
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
# Compute the bootstrap standard error for alpha
alpha_SE = boot_SE(alpha_func,
                   data3,
                   B=2000,
                   seed=456) #bootstrap standard error
alpha_SE
print(f"The bootstrap standard error of the correlation coefficient: {alpha_SE:.4f}")

The bootstrap standard error of the correlation coefficient: 0.0004


In [23]:
# Compute correlation coefficient on the full Default dataset
def alpha_full(data):
    cov_ = np.cov(data[['balance', 'income']], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
            (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))

alpha_actual = alpha_full(data3)
print(f"Actual correlation coefficient (alpha) on full dataset: {alpha_actual:.4f}")

Actual correlation coefficient (alpha) on full dataset: 0.9932


5.Bootstrap vs. Traditional Standard Errors

In [2]:
# load Wage data
data4 =load_data('Wage')

In [5]:
# Function to compute bootstrap SE
def boot_OLS(model_matrix, response, D, idx): #function to compute OLS coefficients
    D_ = D.loc[idx] #bootstrap sample
    Y_ = D_[response] #response variable
    X_ = clone(model_matrix).fit_transform(D_) #design matrix
    return sm.OLS(Y_, X_).fit().params  #return OLS coefficients

wage_func = partial(boot_OLS, MS(['age']), 'wage')
wage_se = boot_SE(wage_func,
                data4,
                B=1500,
                seed=789) #bootstrap standard error
wage_se

intercept    2.563663
age          0.061777
dtype: float64

In [6]:
# Function to compute OLS SE  
wage_model = sklearn_sm(sm.OLS,
                      MS(['age']))
wage_model.fit(data4, data4['wage']) #fit model on full dataset
model_se = summarize(wage_model.results_)['std err']  #extract standard errors
model_se

intercept    2.846
age          0.065
Name: std err, dtype: float64

In [7]:
# Bootstrap standard error for quadratic model
quad_model = MS([poly('age', 2, raw=True)]) #quadratic model
quad_func = partial(boot_OLS,
                    quad_model,
                    'wage') #partial function application
boot_ses=boot_SE(quad_func, data4, B=1500, seed=789)
boot_ses

intercept                           6.138718
poly(age, degree=2, raw=True)[0]    0.313841
poly(age, degree=2, raw=True)[1]    0.003725
dtype: float64

In [8]:
# Fit quadratic model to extract standard errors
M = sm.OLS(data4['wage'],
           quad_model.fit_transform(data4)) #fit quadratic model using statsmodels
ols_ses = summarize(M.fit())['std err']
ols_ses

intercept                           8.190
poly(age, degree=2, raw=True)[0]    0.389
poly(age, degree=2, raw=True)[1]    0.004
Name: std err, dtype: float64

In [41]:
# Calculate ratios (bootstrap SE / OLS SE)
ratios = boot_ses / ols_ses

# Create nice output with coefficient names
coef_names = ['Intercept', 'age', 'age²']
for name, ratio in zip(coef_names, np.round(ratios, 3)):
    print(f"SE ratio for {name:<9}: {ratio:.3f}")

SE ratio for Intercept: 0.750
SE ratio for age      : 0.807
SE ratio for age²     : 0.931
